In [1]:
# data comes from http://web.stanford.edu/~ericyi/project_page/part_annotation/index.html
from pyntcloud import PyntCloud
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
import numpy as np

import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/PartAnnotation"
else:
    data_path = os.getcwd() + "\\PartAnnotation"

# Load model

In [2]:
def get_all_var_names():
    all_vars = []
    for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
        all_vars.append(i.name)
    return all_vars

def get_all_placeholders():
    return [x for x in tf.get_default_graph().get_operations() if x.type == "Placeholder"]

def get_all_mean_op():
    return [x for x in tf.get_default_graph().get_operations() if x.type == "Mean"]

In [3]:
new_graph = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True) # allow passing gpu-trained model to a cpu machine
with tf.Session(graph=new_graph, config=config) as sess:

    import os
    model_fqn = os.path.join(os.getcwd(), 'trained_model', "model-4")
    saver = tf.train.import_meta_graph(model_fqn + ".meta")
    saver.restore(sess, model_fqn)

    graph = tf.get_default_graph()
    print(get_all_var_names())
    print("\nOptimizer", graph.get_collection("optimizer"))
    print("\nPlaceholders", get_all_placeholders())
    print("\nMean", get_all_mean_op())


['conv1/kernel:0', 'conv1/bias:0', 'conv2/kernel:0', 'conv2/bias:0', 'conv4/kernel:0', 'conv4/bias:0', 'conv5/kernel:0', 'conv5/bias:0', 'conv7/kernel:0', 'conv7/bias:0', 'conv8/kernel:0', 'conv8/bias:0', 'bn/beta:0', 'bn/gamma:0', 'bn/moving_mean:0', 'bn/moving_variance:0', 'full_con/kernel:0', 'full_con/bias:0', 'y_pred/kernel:0', 'y_pred/bias:0', 'training/beta1_power:0', 'training/beta2_power:0', 'conv1/kernel/Adam:0', 'conv1/kernel/Adam_1:0', 'conv1/bias/Adam:0', 'conv1/bias/Adam_1:0', 'conv2/kernel/Adam:0', 'conv2/kernel/Adam_1:0', 'conv2/bias/Adam:0', 'conv2/bias/Adam_1:0', 'conv4/kernel/Adam:0', 'conv4/kernel/Adam_1:0', 'conv4/bias/Adam:0', 'conv4/bias/Adam_1:0', 'conv5/kernel/Adam:0', 'conv5/kernel/Adam_1:0', 'conv5/bias/Adam:0', 'conv5/bias/Adam_1:0', 'conv7/kernel/Adam:0', 'conv7/kernel/Adam_1:0', 'conv7/bias/Adam:0', 'conv7/bias/Adam_1:0', 'conv8/kernel/Adam:0', 'conv8/kernel/Adam_1:0', 'conv8/bias/Adam:0', 'conv8/bias/Adam_1:0', 'bn/beta/Adam:0', 'bn/beta/Adam_1:0', 'bn/ga

In [4]:
def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs. (We will only take x,y,z into account for now)
    dim: dimensioin of output voxelized data
    
    This function will locate the grid cube and calculate the density of each cube.
    The output will be normalized values.
    """
    assert(pts.shape[1]>=3), "pts file should contain at least x,y,z coordinate"
    assert(len(dim)==3), "Please provide 3-d grid size like [32,32,32]"
    
    # move all the axis to positive area.
    minimum_val = [pts[0][0], pts[0][1], pts[0][2]]

    # find the smallest 
    for pair in pts:
        if pair[0] < minimum_val[0]:
            minimum_val[0] = pair[0]
        if pair[1] < minimum_val[1]:
            minimum_val[1] = pair[1]
        if pair[2] < minimum_val[2]:
            minimum_val[2] = pair[2]
            
    # move it to first quadrant 
    rectified_pts = np.empty(pts.shape)
    for index, pair in enumerate(pts):
        point = np.zeros(3)
        point[0] = pair[0] - minimum_val[0]
        point[1] = pair[1] - minimum_val[1]
        point[2] = pair[2] - minimum_val[2]
        rectified_pts[index] = point
    
    # biggest value in each axis 
    maximum_val = pts[0][0]
    
    for pair in rectified_pts:
        for val in pair:
            if val > maximum_val:
                maximum_val = val
     
    # normalize all the axises to (0,1)
    normalized_pts = rectified_pts/maximum_val
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2]))
    
    epsilon = 0.000000000001 # we will have at least a 1.0 value which will exceed the index of grid
    # we can use a relativly small value to escape that to fit our data
    
    max_volume_size = 0
    
    for pair in normalized_pts:
        x_loc = int(pair[0]/(x_grid_length + epsilon))
        y_loc = int(pair[1]/(y_grid_length + epsilon))
        z_loc = int(pair[2]/(z_grid_length + epsilon))
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
        
        if output[x_loc, y_loc, z_loc] > max_volume_size:
            max_volume_size = output[x_loc, y_loc, z_loc]
    
    output = output/max_volume_size    
            
    return output

In [5]:
def get_data(data_path, max_file_num=None, dim=[32,32,32]):
    data = []
    
    target_dir_path = os.path.join(data_path, 'points')
    path, dirs, files = os.walk(target_dir_path).__next__()
    file_count = len(files)
    
    count = 0
    for pts_data in os.scandir(target_dir_path):
        if (max_file_num is None) or (count < max_file_num):
            _path = os.path.join(data_path, 'points', pts_data.name)
            pts = PyntCloud.from_file(_path, sep=" ", header=0, names=["x","y","z"])
            _vox = voxelize3D(pts.xyz, dim=dim)
            vox_chan = np.array(_vox).reshape(_vox.shape + (1,))
            data.append(vox_chan)
            count += 1
        else:
            break
                
    return data

In [33]:
def conv3d(_input, w, b, layer=None):
    return tf.nn.relu(tf.nn.conv3d(_input, w, strides=[1, 1, 1, 1, 1], padding="SAME", name="conv" + str(layer)) + b, name="relu" + str(layer))

In [46]:
def get_model_params(model_path):
    
    new_graph = tf.Graph()
    config = tf.ConfigProto(allow_soft_placement=True) # allow passing gpu-trained model to a cpu machine
    
    with tf.Session(graph=new_graph, config=config) as sess:
        
        # Restore model
        saver = tf.train.import_meta_graph(model_path + ".meta")
        saver.restore(sess, model_path)

        model_graph = tf.get_default_graph()
        
        _conv1_w = model_graph.get_tensor_by_name('conv1/kernel:0')
        _conv1_b = model_graph.get_tensor_by_name('conv1/bias:0')   

        _conv2_w = model_graph.get_tensor_by_name('conv2/kernel:0')
        _conv2_b = model_graph.get_tensor_by_name('conv2/bias:0')  

        _conv4_w = model_graph.get_tensor_by_name('conv4/kernel:0')
        _conv4_b = model_graph.get_tensor_by_name('conv4/bias:0')

        _conv5_w = model_graph.get_tensor_by_name('conv5/kernel:0')
        _conv5_b = model_graph.get_tensor_by_name('conv5/bias:0')
        
        _conv7_w = model_graph.get_tensor_by_name('conv7/kernel:0')
        _conv7_b = model_graph.get_tensor_by_name('conv7/bias:0')
        
        _conv8_w = model_graph.get_tensor_by_name('conv8/kernel:0')
        _conv8_b = model_graph.get_tensor_by_name('conv8/bias:0')
    
        _dict = {
            'conv1_w': _conv1_w.eval(),
            'conv1_b': _conv1_b.eval(),
            'conv2_w': _conv2_w.eval(),
            'conv2_b': _conv2_b.eval(),
            'conv4_w': _conv4_w.eval(),
            'conv4_b': _conv4_b.eval(),
            'conv5_w': _conv5_w.eval(),
            'conv5_b': _conv5_b.eval(),
            'conv7_w': _conv7_w.eval(),
            'conv7_b': _conv7_b.eval(),
            'conv8_w': _conv8_w.eval(),
            'conv8_b': _conv8_b.eval(),
        }

    return _dict
    

In [47]:
def cnn3d_model_with_default(params, x_input, output_label_size, keep_rate=0.5):
    
    with tf.name_scope('conv1_layer'):
        conv1_w = tf.Variable(initial_value=params['conv1_w'], name='w')
        conv1_b = tf.Variable(initial_value=params['conv1_b'], name='b')

        conv1 = conv3d(x_input, conv1_w, conv1_b, 1)

    with tf.name_scope('conv2_layer'):
        conv2_w = tf.Variable(initial_value=params['conv2_w'], name='w')
        conv2_b = tf.Variable(initial_value=params['conv2_b'], name='b')

        conv2 = conv3d(conv1, conv2_w, conv2_b, 2)

    with tf.name_scope('pool3_layer'):
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2, name="pool3")


    with tf.name_scope('conv4_layer'):
        conv4_w = tf.Variable(initial_value=params['conv4_w'], name='w')
        conv4_b = tf.Variable(initial_value=params['conv4_b'], name='b')
      
        conv4 = conv3d(pool3, conv4_w, conv4_b, 4)
        
    with tf.name_scope('conv5_layer'):
        conv5_w = tf.Variable(initial_value=params['conv5_w'], name='w')
        conv5_b = tf.Variable(initial_value=params['conv5_b'], name='b')
      
        conv5 = conv3d(conv4, conv5_w, conv5_b, 5)
        
    with tf.name_scope('pool6_layer'):
        pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2, name="pool6")
        
    with tf.name_scope('conv7_layer'):
        conv7_w = tf.Variable(initial_value=params['conv7_w'], name='w')
        conv7_b = tf.Variable(initial_value=params['conv7_b'], name='b')
      
        conv7 = conv3d(pool6, conv7_w, conv7_b, 7)
        
    with tf.name_scope('conv8_layer'):
        conv8_w = tf.Variable(initial_value=params['conv8_w'], name='w')
        conv8_b = tf.Variable(initial_value=params['conv8_b'], name='b')
      
        conv8 = conv3d(conv7, conv8_w, conv8_b, 8)
        
    with tf.name_scope('pool9_layer'):
        pool9 = tf.layers.max_pooling3d(inputs=conv8, pool_size=[2, 2, 2], strides=2, name="pool9")
           
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=pool9, training=True, name="bn")
        
    with tf.name_scope("fully_con"):
        flattening = tf.reshape(cnn3d_bn, [-1, 4*4*4*512])
        dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu, name="full_con")
        # (1-keep_rate) is the probability that the node will be kept
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True, name="dropout")
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=output_label_size, name="y_pred") 
        
    return y_conv

In [64]:
# one hot indexes
# 2: chair , 3: car

import os
model_path = os.path.join(os.getcwd(), 'model', 'model-0')

params = get_model_params(model_path)

learning_rate = 0.005

a = get_data(data_path + "/table", max_file_num=10)
y = np.zeros((10,5))

for index, _ in enumerate(y):
    y[index][4] = 1
print(y)
    
with tf.Session(graph=tf.Graph()) as sess:
    
    with tf.name_scope('inputs'):
        x_input = tf.placeholder(tf.float32, shape=[None, 32, 32, 32, 1], name="x_input")
        y_input = tf.placeholder(tf.float32, shape=[None, 5], name="y_input") 

    prediction = cnn3d_model_with_default(params, x_input, 5)
    tf.add_to_collection("logits", prediction)

    with tf.name_scope("cross_entropy"):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input), name="cross_entropy")

    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        tf.add_to_collection("optimizer", optimizer)

    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name="acc")
    
    
    sess.run(tf.global_variables_initializer())
    
    stop_point = tf.get_default_graph().get_tensor_by_name('conv5_layer/conv5:0')
    
    stopper = tf.stop_gradient(stop_point, 'stop_gradient')
    
    acc = sess.run([accuracy], feed_dict={x_input: a, y_input: y})
    
    print(acc)
    
     # Runs train_op.
    _ , acc = sess.run([optimizer,accuracy], feed_dict={x_input: a, y_input: y})
    print(acc)

[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]]
[0.0]
0.1
